In [1]:
#imports
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [2]:
#Get the data
data = pd.read_csv(r'C:\Users\Akkshay\Downloads\JEOPARDY_CSV.csv')
data = shuffle(data, random_state = 22)
data.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
208762,2991,1997-09-08,Jeopardy!,AMERICAN HISTORY,$200,In 1890 this GOP senator from Ohio sponsored a...,John Sherman
85442,3040,1997-11-14,Double Jeopardy!,TEENS ON FILM,$200,"In this comedy named for a holiday, Charlie Ho...",Father's Day
17573,731,1987-11-09,Jeopardy!,EUROPE ON FILM,$400,"Country in which Hitchcock's ""Jamaica Inn"" was...",Britain
147747,4293,2003-04-09,Double Jeopardy!,"THE ""HOLE"" THING",$2000,These heavenly phenomena are defined in part b...,black holes
278,4931,2006-02-06,Double Jeopardy!,NOT A CURRENT NATIONAL CAPITAL,$1200,"Sofia, Sarajevo, Saigon",Saigon


In [7]:
data = data.drop(data.columns[2], axis = 1)
data.head()

,Category,Question
208762,AMERICAN HISTORY,In 1890 this GOP senator from Ohio sponsored a...
85442,TEENS ON FILM,"In this comedy named for a holiday, Charlie Ho..."
17573,EUROPE ON FILM,"Country in which Hitchcock's ""Jamaica Inn"" was..."
147747,"THE ""HOLE"" THING",These heavenly phenomena are defined in part b...
278,NOT A CURRENT NATIONAL CAPITAL,"Sofia, Sarajevo, Saigon"


In [25]:
#Encode categories to multi-hot 
#cat_split = [cat for cat in data[data.columns[0]]]
#print(cat_split, "\n")
cat_split = []
############have a for loop that adds the cat to cat_split

for cat in data[data.columns[0]]:
    #print(cat)
    cat_split.append(cat)

#print(cat_split)
cat_encoder = MultiLabelBinarizer()
cats_encoded = cat_encoder.fit_transform(cat_split)
num_cats = len(cats_encoded[0])
print(data[data.columns[1]].values[0])
print(cat_encoder.classes_)
print(cats_encoded[0])

In 1890 this GOP senator from Ohio sponsored an antitrust act & the Silver Purchase Act
[' ' '!' '"' '$' '&' "'" '(' ')' '*' '+' ',' '-' '.' '/' '0' '1' '2' '3'
 '4' '5' '6' '7' '8' '9' ':' '=' '?' '@' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'k' 'l' 'm' 'n' 'o' 'p' 'r' 's'
 't' 'u' 'v' 'w' 'x' 'y' '¡' '¿' 'À' 'Ç' 'É' 'Í' 'Ñ' '–' '‘' '’' '“' '”'
 '…' '♥']
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1
 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [26]:
#test test split 80/20
train_size = int(len(data)*.8)
print("Train_size: ", train_size)
print("Test _size: ", (len(data)-train_size))

Train_size:  173544
Test _size:  43386


In [34]:
#splitting the data
train_cat = cats_encoded[:train_size]
test_cat = cats_encoded[train_size:]

In [36]:
%%writefile preprocess.py
#create tokenizer class
from tensorflow.keras.preprocessing import text

class TextProcessor(object):
    def __init__(self, vocab_size):
        self._vocab_size = vocab_size
        self._tokenizer = None
    def create_tokenizer(self, text_list):
        tokenizer = text.Tokenizer(num_words = self._vocab_size)
        tokenizer.fit_on_texts(text_list)
        self._tokenizer = tokenizer
    def transform_text(self, text_list):
        text_matrix = self._tokenizer.texts_to_matrix(text_list)
        return text_matrix

Writing preprocess.py


In [37]:
#create bag of words matrices
from preprocess import TextProcessor

train_qs = data[data.columns[1]].values[:train_size]
test_qs = data[data.columns[1]].values[train_size:]
VOCAB_SIZE = 800

processor = TextProcessor(VOCAB_SIZE)
processor.create_tokenizer(train_qs)

body_train = processor.transform_text(train_qs)
body_test = processor.transform_text(test_qs)

In [38]:
#preview out training data
print(len(body_train[0]))
print(body_train[0])

800
[0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [39]:
#save the tokenizer state
import pickle
with open('./processor_state.pkl', 'wb') as f:
    pickle.dump(processor, f)

In [43]:
#create model

def create_model(vocab_size, num_cats):
    model = Sequential()
    model.add(Dense(50, input_shape = (vocab_size,), activation = 'relu'))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(num_cats, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = create_model(VOCAB_SIZE, num_cats)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                40050     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 92)                4692      
Total params: 47,292
Trainable params: 47,292
Non-trainable params: 0
_________________________________________________________________


In [44]:
#train
model.fit(body_train, train_cat, epochs = 3, batch_size = 128, validation_split = 0.1)

Train on 156189 samples, validate on 17355 samples
Epoch 1/3
156189/156189 [==============================] - 48s 304us/sample - loss: 0.1940 - accuracy: 0.9152 - val_loss: 0.1734 - val_accuracy: 0.9204
Epoch 2/3
156189/156189 [==============================] - 27s 172us/sample - loss: 0.1729 - accuracy: 0.9205 - val_loss: 0.1723 - val_accuracy: 0.9209
Epoch 3/3
156189/156189 [==============================] - 23s 145us/sample - loss: 0.1718 - accuracy: 0.9209 - val_loss: 0.1718 - val_accuracy: 0.9211


In [46]:
#evaluate
model.evaluate(body_test, test_cat, batch_size = 128)

43386/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.1719102644061161, 0.92098206]

In [47]:
model.save('keras_jeoprady_QClassification_saved_model.h5')

In [ ]:
##### write custom prediction class